<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter13/chapter13_7_Project2_MNIST_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13.7 프로젝트2 : MNIST 손글씨 숫자 분류하기
이번 절에서는 MNIST데이터셋의 손글씨 숫자를 분류하는 프로젝트를 진행해보자

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'Colab Notebooks/pyml/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/pyml


In [2]:
# 1. 설정단계
# 데이터셋을 로드하고 하이퍼파라미터(훈련 세트와 테스트 세트 크기, 미니 배치 크기)를 지정하자
# torchvision 모듈을 사용하여 MNISt데이터셋을 로드하자
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import DataLoader, TensorDataset
image_path = 'Chapter13/'
transform = transforms.Compose([
    transforms.ToTensor() #
])

mnist_train_dataset = torchvision.datasets.MNIST(
    root = image_path, train = True,
    transform = transform, download = True)

mnist_test_dataset = torchvision.datasets.MNIST(
    root = image_path, train = False,
    transform = transform, download = False
)

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset,
                      batch_size, shuffle = True)



## 2. 입력 특성과 레이블 전처리 단계
이 프로젝트에서 데이터의 특성은 1단계에서 읽은 이밎의 픽셀임.  
`torchvision.transforms.Compose`를 사용하여 사용자 정의 변환을 정의할 수 있음.  
간단한 이 예제에서는 `ToTensor()` 메서드 하나로 변환이 구성됨.  
`ToTensor()` 메서드는 픽셀 특성을 실수 텐서로 변환하고 [0,255] 범위의 픽셀을 [0,1] 범위로 정규화함.  
14장에서 더 복잡한 이미지 데이터셋을 다룰 떄 몇 가지 추가적인 데이터 변환 방법을 살펴보자.  

레이블의 경우 0에서 9까지의 정수이므로, 추가적인 스케일 조정이나 변환이 필요하지 않음.  

`data`속성을 사용하여 원시 픽셀 값을 참조할 수 있으며, [0, 1]범위로 변환되었다는 것을 잊지 말자.  

In [3]:
print(mnist_train_dataset[0][0].shape)

torch.Size([1, 28, 28])


In [7]:
# 3. 신경망 모델 만들기
import torch.nn as nn
hidden_units = [32,16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]
all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
  layer = nn.Linear(input_size, hidden_unit)
  all_layers.append(layer)
  all_layers.append(nn.ReLU())
  input_size = hidden_unit
all_layers.append(nn.Linear(hidden_units[-1],10))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

In [8]:
# 4.  모델 사용하여 훈련, 평가 및 예측하기
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
torch.manual_seed(1)
num_epochs = 20

for epoch in range(num_epochs):
  accuracy_hist_train = 0
  for x_batch, y_batch in train_dl:
    pred = model(x_batch)
    loss = loss_fn(pred, y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    is_correct = (
        torch.argmax(pred, dim = 1) == y_batch
    ).float()
    accuracy_hist_train += is_correct.sum()
  accuracy_hist_train /= len(train_dl.dataset)
  print(f'epoch : {epoch} accuracy '
        f'{accuracy_hist_train:.4f}')

epoch : 0 accuracy0.8576
epoch : 1 accuracy0.9295
epoch : 2 accuracy0.9441
epoch : 3 accuracy0.9527
epoch : 4 accuracy0.9577
epoch : 5 accuracy0.9617
epoch : 6 accuracy0.9657
epoch : 7 accuracy0.9676
epoch : 8 accuracy0.9691
epoch : 9 accuracy0.9721
epoch : 10 accuracy0.9734
epoch : 11 accuracy0.9752
epoch : 12 accuracy0.9766
epoch : 13 accuracy0.9776
epoch : 14 accuracy0.9786
epoch : 15 accuracy0.9805
epoch : 16 accuracy0.9811
epoch : 17 accuracy0.9826
epoch : 18 accuracy0.9826
epoch : 19 accuracy0.9839


In [9]:
# 5. 테스트 세트 평가
pred = model(mnist_test_dataset.data / 255.)
is_correct = (
    torch.argmax( pred , dim = 1) == mnist_test_dataset.targets
).float()

print(f'테스트 정확도: {is_correct.mean():.4f}')

테스트 정확도: 0.9667
